<a href="https://colab.research.google.com/github/Datamin-D/TIL/blob/master/%5BAbsa_2_4%5D_Lexicon_base_Aspect_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 선행 연구의 Lexicon based aspect classification

## Load data * libraries

In [2]:
# from top2vec import Top2Vec
from google.colab import drive
drive.mount("/gdrive")

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [4]:
import os
import numpy as np
import pandas as pd
path = '/gdrive/MyDrive/ABSA2 Multi labeling/lexicon based'
os.chdir(path)
aspects = pd.read_excel('Lexicon_aspects.xlsx') # excel
# aspects = pd.read_csv('brand asepct.csv') # csv
aspects

,movie,actor,directing,plot,Music
0,movie,act,direct,storyline,lyric
1,film,acting,direction,story,sound
2,NaN,actress,directing,tale,music
3,NaN,actor,director,romance,audio
4,NaN,role,filmed,dialog,musical
5,NaN,portray,filming,script,title track
6,NaN,character,filmmaking,storyteller,sound effect
7,NaN,villain,filmmaker,ending,sound track
8,NaN,performance,cinematic,storytelling,NaN
9,NaN,performed,edition,revenge,NaN


In [13]:
import os
path = "/gdrive/MyDrive/movie_predict/ABSA/ABSA2 Multi labeling/실제 코드/labeled_data/labeled_all"
os.chdir(path)
import pickle
with open("absa2data.pkl",'rb') as fr:
    df = pickle.load(fr)
df

,Unnamed: 0.1,sent_id,sent,actor,plot,music,directing,movie,none
0,9720,5396910,it provokes a sense of a revolution within you...,0,1.0,0,0,2,0
1,9721,57011,an incredibly well-balanced and viscerally stu...,0,1.0,0,2,0,0
2,9722,2622250,i cannot stress enough that this movie could h...,1,0.0,0,0,0,0
3,9723,6151486,why is he like iron man all of the sudden?!?,0,1.0,0,0,0,0
4,9724,5834136,something like that would not have even been d...,0,1.0,0,0,2,0
...,...,...,...,...,...,...,...,...,...
38873,9715,7057912,i only hope that matt will do another and they...,1,0.0,0,0,0,0
38874,9716,1158436,didn't read the book.,0,0.0,0,0,0,1
38875,9717,4604373,the whole concept of time travel was obviously...,0,1.0,0,0,0,0
38876,9718,1683902,"can we have dumbo next, please?",0,0.0,0,0,0,1


#### Preprocessing 이미함 pass

In [21]:
df = df.dropna()
df.head()

,Unnamed: 0.1,sent_id,sent,actor,plot,music,directing,movie,none
0,9720,5396910,it provokes a sense of a revolution within you...,0,1.0,0,0,2,0
1,9721,57011,an incredibly well-balanced and viscerally stu...,0,1.0,0,2,0,0
2,9722,2622250,i cannot stress enough that this movie could h...,1,0.0,0,0,0,0
3,9723,6151486,why is he like iron man all of the sudden?!?,0,1.0,0,0,0,0
4,9724,5834136,something like that would not have even been d...,0,1.0,0,0,2,0


In [23]:
asp_list = ['actor', 'plot', 'music','directing', 'movie'] # df의 aspects

def aspect2one(x):
  if x == 0:
    return 0
  else:
    return 1

for aspect in asp_list:
  df[aspect] = df[aspect].apply(lambda x: int(x))
  # df[aspect].value_counts()
  
  df[aspect] = df[aspect].apply(lambda x: aspect2one(x))

In [ ]:
df # multi-hot encoding 됨

In [25]:
# lower
from tqdm import tqdm
tqdm.pandas()

def lower(sent):
  try:
    return str(sent).lower()
  except:
    print(sent)
df['sent'] = df['sent'].progress_apply(lambda sent: lower(sent))
df['sent'].head()

100%|██████████| 38876/38876 [00:00<00:00, 555936.22it/s]


0    it provokes a sense of a revolution within you...
1    an incredibly well-balanced and viscerally stu...
2    i cannot stress enough that this movie could h...
3         why is he like iron man all of the sudden?!?
4    something like that would not have even been d...
Name: sent, dtype: object

# actor

## train test split

In [41]:
# 이미 나뉜 train/test split #lower되어있음.
aspect = 'actor'
os.chdir('/gdrive/MyDrive/movie_predict/ABSA/ABSA2 Multi labeling/실제 코드/labeled_data/labeled_all')

with open('train_'+aspect,'rb') as fr:
  train_data = pickle.load(fr)

with open('test_'+aspect,'rb') as fr:
  test_data = pickle.load(fr)

In [ ]:
test_data

## Aspect Categorization w Lexicon

In [67]:
# 여러개의 aspect를 classification 동시에 할 때 코드, 존재하면 aspect 이름을 return

from tqdm import tqdm
tqdm.pandas()

# 주의. 아래 코드는 적당한 단어 길이에서만 통함. 두세글자 단어들이 들어가는 경우 \b 코드를 정규 표현식으로 써야함.
#왜? 아래 코드는 aspect term 에 'cat' 이 있을경우 'categorization'도 인식함. 본 task에는 적합한 code.
def classify_aspect(sent,aspect):
  aspect_terms = aspects[aspect].dropna()
  aspect_terms = list(aspect_terms)
  for aspect_term in aspect_terms: # aspect term 반복
    if aspect_term in sent:
      return 1 # aspect term 있으면 aspect return
    else:
      return 0
# temp = df.head(500)
for aspect in list(aspects.columns):
  # temp["pred_"+aspect] = temp['sent'].progress_apply(lambda sent: classify_aspect(sent,aspect))
  test_data["pred_"+aspect] = test_data['sent'].progress_apply(lambda sent: classify_aspect(sent,aspect))

100%|██████████| 7776/7776 [00:00<00:00, 9169.36it/s]


In [68]:
# 한개의 aspect를 classification할 때 코드
# 여러개의 aspect를 classification 동시에 할 때 코드
aspect = 'actor'
from tqdm import tqdm
tqdm.pandas()

# 주의. 아래 코드는 적당한 단어 길이에서만 통함. 두세글자 단어들이 들어가는 경우 \b 코드를 정규 표현식으로 써야함.
#왜? 아래 코드는 aspect term 에 'cat' 이 있을경우 'categorization'도 인식함. 본 task에는 적합한 code.
def classify_aspect(sent,aspect):
  aspect_terms = aspects[aspect].dropna()
  aspect_terms = list(aspect_terms)
  for aspect_term in aspect_terms: # aspect term 반복
    if aspect_term in sent:
      return 1 # aspect term 있으면 aspect return
    else:
      return 0
# temp = df.head(500)
# temp["pred_"+aspect] = temp['sent'].progress_apply(lambda sent: classify_aspect(sent,aspect))
test_data["pred_"+aspect] = test_data['sent'].progress_apply(lambda sent: classify_aspect(sent,aspect))

100%|██████████| 7776/7776 [00:00<00:00, 8420.86it/s]


### Evaluation

In [69]:
import numpy as np
test_ans = np.array(test_data[aspect])
test_pred = np.array(test_data['pred_'+aspect].fillna(0))

In [71]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

cm = confusion_matrix(test_ans, test_pred)
print (cm)
print ('\n Here is the classification report:')
print (classification_report(test_ans, test_pred))


[[5651 1043]
 [ 367  715]]

 Here is the classification report:
              precision    recall  f1-score   support

           0       0.94      0.84      0.89      6694
           1       0.41      0.66      0.50      1082

    accuracy                           0.82      7776
   macro avg       0.67      0.75      0.70      7776
weighted avg       0.86      0.82      0.84      7776



In [ ]:
# 이렇게 다른 aspect 도 진행.